![image](https://www.python.org/static/img/python-logo.png)
# 亞洲大學基礎程式設計教材(AUP110-Fundamentals of Programming)
![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)


# Week17-網址工具(urllib)、資料庫(Database)、網路爬蟲(Network)、非同步執行(asynchronous)






## Topic 1(主題1)-urllib函數庫
https://docs.python.org/3/howto/urllib2.html

urllib.request 是一個用來從URLs (Uniform Resource Locators)取得資料的Python模組。它提供一個了非常簡單的介面能接受多種不同的協議， urlopen 函數。也提供了較複雜的介面用於處理一些常見的狀況，例如:基本的authentication、cookies、proxies等等，這些都可以由handler或opener物件操作。

一般情形下 urlopen 是非常容易使用的，但當你遇到錯誤或者較複雜的情況下，你可能需要對超文本協議HyperText Transfer Protocol有一定的了解。最完整且具參考價值的是 RFC 2616，不過它是一份技術文件並不容易閱讀，以下的教學會提供足夠的HTTP知識來幫助你使用 urllib。這份教學並非要取代 urllib.request 這份文件，你還是會需要它!

### Step 1: 從URL取得資源

In [ ]:
import urllib.request
with urllib.request.urlopen('http://python.org/') as response:
   html = response.read()

### Step 2:異常的處理
* URLError: 如果 urlopen 無法處理響應信息，就會觸發 URLError 。
* HTTPError 是 URLError 的子類，當 URL 是 HTTP 的情況時將會觸發。



In [ ]:
#方案1
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
req = Request(someurl)
try:
    response = urlopen(req)
except HTTPError as e:
    print('The server couldn\'t fulfill the request.')
    print('Error code: ', e.code)
except URLError as e:
    print('We failed to reach a server.')
    print('Reason: ', e.reason)
else:
    # everything is fine

In [ ]:
#方案2
from urllib.request import Request, urlopen
from urllib.error import URLError
req = Request(someurl)
try:
    response = urlopen(req)
except URLError as e:
    if hasattr(e, 'reason'):
        print('We failed to reach a server.')
        print('Reason: ', e.reason)
    elif hasattr(e, 'code'):
        print('The server couldn\'t fulfill the request.')
        print('Error code: ', e.code)
else:
    # everything is fine

## Topic 2(主題2)-SQLite 數據庫 DB-API 2.0 接口
https://docs.python.org/3/library/sqlite3.html


### Step 3:創建一個 Connection 物件
要使用SQLite，必須先創建一個 Connection 物件，它代表數據庫。

In [ ]:
import sqlite3
con = sqlite3.connect('example.db')

### Step 4:創建一個 Cursor 游標物件
當有了 Connection 對像後，你可以創建一個 Cursor 游標物件，然後調用它的 execute() 方法來執行 SQL 語句：

In [ ]:
cur = con.cursor()

# Create table
cur.execute('''CREATE TABLE stocks
               (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
cur.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
con.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
con.close()

### Step 5:執行 SELECT SQL查詢後獲取數據

要在執行 SELECT 語句後獲取數據，你可以把游標作為 iterator，然後調用它的 fetchone() 方法來獲取一條匹配的行，也可以調用 fetchall() 來得到包含多個匹配行的列表。

In [ ]:
import sqlite3
con = sqlite3.connect('example.db')
cur = con.cursor()
for row in cur.execute('SELECT * FROM stocks ORDER BY price'):
  print(row)

### Step 6:
應該使用的是 DB-API 的形參替換。在任何你要使用值的地方放一個佔位符，然後提供包含多個值的元組作為數據游標的 execute() 方法的第二個參數。 SQL 語句可以使用兩種類別的佔位符之一：問號佔位符（問號風格）或名稱佔位符（名稱風格）。對於問號風格，parameters 必須為 序列。對於名稱風格，它可以為 序列 或 dict 實例。序列 的長度必須匹配佔位符的數量，否則會引發 ProgrammingError。如果給出的是 dict，則它必須包含與全部名稱形參對應的鍵。任何額外條目會被忽略。下面是包含這兩種風格的例子:

In [ ]:
import sqlite3

con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.execute("create table lang (name, first_appeared)")

# This is the qmark style:
cur.execute("insert into lang values (?, ?)", ("C", 1972))

# The qmark style used with executemany():
lang_list = [
    ("Fortran", 1957),
    ("Python", 1991),
    ("Go", 2009),
]
cur.executemany("insert into lang values (?, ?)", lang_list)

# And this is the named style:
cur.execute("select * from lang where first_appeared=:year", {"year": 1972})
print(cur.fetchall())

con.close()

### Step 7: sqlite3 的時間處理
sqlite3 模塊有兩個適配器可用於 Python 的內置 datetime.date 和 datetime.datetime 類型。

In [ ]:
import sqlite3
import datetime
import time

def adapt_datetime(ts):
    return time.mktime(ts.timetuple())

sqlite3.register_adapter(datetime.datetime, adapt_datetime)

con = sqlite3.connect(":memory:")
cur = con.cursor()

now = datetime.datetime.now()
cur.execute("select ?", (now,))
print(cur.fetchone()[0])

con.close()

## Topic 3(主題3)-收集學校新聞的大數據


### Step 8: 讀取學校的新聞標題

In [ ]:
from urllib import request
with request.urlopen('http://www.asia.edu.tw/news1.php') as response:
    html = response.read().decode('utf-8')
    print(html)

### Step 9: 每一年的新聞標題

In [ ]:
import re
from urllib import request
count = 0
sss = ["2008", "2009","2010", "2011", "2012","2013","2014","2015","2016","2017","2018" ,"2019","2020", "2021"]
titles=list()
for i in range(len(sss)):
    year = sss[i]
    with request.urlopen('http://www.asia.edu.tw/news1.php?y='+year) as response:
        html = response.read().decode('utf-8')
        #print(html)
        pattern = '<font color="#446666" face="新細明體" style="font-weight: 700;" size="2">'
        for pos in re.finditer(pattern, html):
            pos2 = html.find('</font>', pos.end())
            sub = html[pos.end():pos2]
            titles.append(sub)
            count = count + 1
print (count)

### Step 10: 建立新聞標題的資料庫



In [ ]:
import sqlite3
conn = sqlite3.connect('news.db')
c = conn.cursor()

# Create table
c.execute("CREATE TABLE news (title text)")

# Insert a row of data
for t in titles:
  ss = "INSERT INTO news VALUES ('{}')".format(t)
  c.execute(ss)

# Save (commit) the changes
conn.commit()
conn.close()

### Step 11: 查詢學校新聞標題有人工智慧

In [ ]:
conn = sqlite3.connect('news.db')
c = conn.cursor()
for row in c.execute('''SELECT * FROM news 
                        WHERE title LIKE '%人工智慧%' '''):
    print(row)
conn.close()

## Topic 4(主題4)-asyncio (Since 3.4) 非同步式(asynchronous)程式

### Step 12:同步的網頁要求
以下範例是常見的程式寫法，該範例在 do_requests() 函式中以 for 迴圈對 example.com 發出 10 次 HTTP GET 要求(request)，並且列印其狀態碼(status code)：

![](https://myapollo.com.tw/images/begin-to-asyncio/seq.png)




In [ ]:
import requests
import time

url = 'https://www.google.com.tw/'

start_time = time.time()

def send_req(url):

    t = time.time()
    print("Send a request at",t-start_time,"seconds.")

    res = requests.get(url)

    t = time.time()
    print("Receive a response at",t-start_time,"seconds.")

for i in range(10):
    send_req(url)

### Step 13:非同步的網頁要求

In [ ]:
!pip install aiohttp requests

In [ ]:
import requests
import time
import asyncio

url = 'https://www.asia.edu.tw/'

start_time = time.time()
async def send_req(url):
    t = time.time()
    print("Send a request at",t-start_time,"seconds.")
    res = await loop.run_in_executor(None,requests.get,url)
    t = time.time()
    print("Receive a response at",t-start_time,"seconds.")
tasks = []

loop = asyncio.get_event_loop()
for i in range(10):
    task = loop.create_task(send_req(url))
    tasks.append(task)
try:
  loop.run_until_complete(asyncio.wait(tasks))
except:
     print("ended!")

##Topic 5(主題5)-並行 Concurrency

多引線執行(multithreading)，

### Step 14:兩個函數在同一個process(thread) 執行。

In [ ]:
import time
def sleep_A():
    for i in range(2):
        print(i, end="_")
        time.sleep(1)
    return
def sleep_B():
    for i in range(3):
        print(i, end="=")
        time.sleep(1)
    return
start_time = time.time()
sleep_A()
sleep_B()
end_time = time.time()
print(f'It costs {end_time - start_time} seconds')

### Step 15:把兩個函數在不同的thread同時進行。

In [ ]:
import os
import threading

def sleep_A():
    for i in range(2):
        print(i, end="_")
        time.sleep(1)
    return

def sleep_B():
    for i in range(3):
        print(i, end="=")
        time.sleep(1)
    return
  
start_time = time.time()

thread_1 = threading.Thread(target=sleep_A)  # 例項化一個執行緒物件，使執行緒執行這個函式
thread_2 = threading.Thread(target=sleep_B)  # 例項化一個執行緒物件，使執行緒執行這個函式
thread_1.start()  # 啟動這個執行緒
thread_2.start()  # 啟動這個執行緒
thread_1.join()  # 等待thread_1結束，如果不打join程式會直接往下執行
thread_2.join()  # 等待thread_2結束，如果不打join程式會直接往下執行

end_time = time.time()
print(f'It costs {end_time - start_time} seconds')

##Topic 6(主題6)--內建函數和函數庫的複習

### Step 16:文字排序

In [ ]:
# 升序排序(ascending sort )
coms = ['Microsoft', 'Google', 'Amazon', 'Facebook', 'Apple']
print(sorted(coms))

In [ ]:
# 降序排序(descending sort )
coms = ['Microsoft', 'Google', 'Amazon', 'Facebook', 'Apple']
print(sorted(coms, reverse=True))

Step 17: 每個月的第一天星期幾和有幾天

In [ ]:
#calendar.monthrange()函數可以知道每個月的第一天星期幾和有幾天
import calendar
from datetime import datetime, timezone, timedelta
# 設定為 +8 時區
tz = timezone(timedelta(hours=+8))
dt=datetime.now(tz)
wday, mdays = calendar.monthrange(dt.year,dt.month)
print(f"For {dt.year}/{dt.month}, weekday is {wday}; number of days is {mdays}") #weekday (0-6 ~ Mon-Sun)